In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install opencv-python

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import cv2

In [4]:
BASE_DIR = '/content/drive/MyDrive/ROAD-CONDITIONS.v1-weather-detection'
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class WeatherDataset(Dataset):
    def __init__(self, img_dir, csv_file, transform=None):
        self.img_dir = img_dir
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.labels_cols = self.data.columns[1:]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        img_path = os.path.join(self.img_dir, row['filename'])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Multi-hot labels
        labels = torch.tensor(row[self.labels_cols].values.astype(float), dtype=torch.float32)
        return image, labels

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

In [7]:
train_dataset = WeatherDataset(os.path.join(BASE_DIR, 'train'), os.path.join(BASE_DIR, 'train', '_classes.csv'), transform)
val_dataset   = WeatherDataset(os.path.join(BASE_DIR, 'valid'), os.path.join(BASE_DIR, 'valid', '_classes.csv'), transform)
test_dataset  = WeatherDataset(os.path.join(BASE_DIR, 'test'), os.path.join(BASE_DIR, 'test', '_classes.csv'), transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

num_classes = len(train_dataset.labels_cols)
print("Label columns:", list(train_dataset.labels_cols))
print("Number of classes:", num_classes)

Label columns: ['Lighting_Dusk', 'Surface_Dry', 'Surface_Unknown', 'Surface_Wet', 'Weather_Clear', 'Weather_Fog', 'Weather_Rain', 'Weather_Unknown']
Number of classes: 8


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
model = models.efficientnet_b1(pretrained=True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
model = model.to(DEVICE)

# Freeze base layers initially
for param in model.features.parameters():
    param.requires_grad = False

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B1_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/efficientnet_b1_rwightman-bac287d4.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1_rwightman-bac287d4.pth


100%|██████████| 30.1M/30.1M [00:00<00:00, 103MB/s]


In [9]:
from tqdm import tqdm

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        model.train()
        running_loss = 0.0

        # Training loop with progress bar
        train_bar = tqdm(train_loader, desc="Training Batches")
        for inputs, labels in train_bar:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            train_bar.set_postfix(loss=loss.item())

        epoch_loss = running_loss / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc="Validation Batches")
            for inputs, labels in val_bar:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                val_bar.set_postfix(val_loss=loss.item())

        val_loss /= len(val_loader.dataset)
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {epoch_loss:.4f} - Val Loss: {val_loss:.4f}")



In [ ]:
# Train
train_model(model, train_loader, val_loader, criterion, optimizer, EPOCHS)


Epoch 1/20


Validation Batches: 100%|██████████| 9/9 [00:34<00:00,  3.86s/it, val_loss=0.228]


Epoch 1/20 - Train Loss: 0.3633 - Val Loss: 0.2614

Epoch 2/20


Validation Batches: 100%|██████████| 9/9 [00:01<00:00,  4.59it/s, val_loss=0.172]


Epoch 2/20 - Train Loss: 0.2209 - Val Loss: 0.2096

Epoch 3/20


...

Epoch 18/20 - Train Loss: 0.1118 - Val Loss: 0.1203

Epoch 19/20


Validation Batches: 100%|██████████| 9/9 [00:01<00:00,  5.35it/s, val_loss=0.0821]


Epoch 19/20 - Train Loss: 0.1089 - Val Loss: 0.1191

Epoch 20/20


Validation Batches: 100%|██████████| 9/9 [00:01<00:00,  6.18it/s, val_loss=0.078]

Epoch 20/20 - Train Loss: 0.1085 - Val Loss: 0.1149


In [ ]:
# Fine-tune last 20 layers
for param in model.features[-20:].parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-5)
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)






Epoch 1/10


Validation Batches: 100%|██████████| 9/9 [00:01<00:00,  5.27it/s, val_loss=0.0613]


Epoch 1/10 - Train Loss: 0.0944 - Val Loss: 0.0984

Epoch 2/10


Validation Batches: 100%|██████████| 9/9 [00:01<00:00,  5.34it/s, val_loss=0.0565]


Epoch 2/10 - Train Loss: 0.0768 - Val Loss: 0.0909

Epoch 3/10


...

Epoch 8/10 - Train Loss: 0.0343 - Val Loss: 0.0667

Epoch 9/10


Validation Batches: 100%|██████████| 9/9 [00:01<00:00,  6.43it/s, val_loss=0.031]


Epoch 9/10 - Train Loss: 0.0300 - Val Loss: 0.0636

Epoch 10/10


Validation Batches: 100%|██████████| 9/9 [00:01<00:00,  4.56it/s, val_loss=0.0357]

Epoch 10/10 - Train Loss: 0.0266 - Val Loss: 0.0634


In [12]:
# =========================================
# Save Model
# =========================================
torch.save(model.state_dict(), os.path.join(BASE_DIR, 'weather_model_efficientnetb1_multilabel.pt'))
print("✅ Model saved!")

✅ Model saved!


In [ ]:
def predict_image_with_accuracy(img_path, true_labels=None, threshold=0.25):
    img = Image.open(img_path).convert('RGB')
    input_tensor = transform(img).unsqueeze(0).to(DEVICE)

    model.eval()
    with torch.no_grad():
        outputs = torch.sigmoid(model(input_tensor))

    outputs_np = outputs.cpu().numpy()[0]

    # Map columns to readable names
    label_map = {
        "Lighting_Dusk": "Dusk",
        "Surface_Dry": "Dry",
        "Surface_Unknown": "Unknown Surface",
        "Surface_Wet": "Wet",
        "Weather_Clear": "Clear",
        "Weather_Fog": "Foggy",
        "Weather_Rain": "Rainy",
        "Weather_Unknown": "Unknown Weather"
    }

    predicted_labels = [label_map[col] for col, val in zip(train_dataset.labels_cols, outputs_np) if val > threshold]

    # Print probabilities for debugging
    print("Predicted probabilities per label:")
    print({label_map[col]: float(val) for col, val in zip(train_dataset.labels_cols, outputs_np)})

    # Compute accuracy if true_labels provided
    if true_labels is not None:
        true_labels = np.array(true_labels)
        pred_binary = (outputs_np > threshold).astype(int)
        accuracy = (pred_binary == true_labels).mean()
        print(f"✅ Accuracy for this image: {accuracy*100:.2f}%")

    return predicted_labels


In [ ]:

true_labels = [0,0,0,0,1,0,0,0]

preds = predict_image_with_accuracy('/content/fog.jpeg', true_labels=true_labels)
print("Predicted Labels:", preds)


Predicted probabilities per label:
{'Dusk': 0.0004811091930605471, 'Dry': 0.8592066168785095, 'Unknown Surface': 0.0033211810514330864, 'Wet': 0.08461613953113556, 'Clear': 0.11182582378387451, 'Foggy': 0.8730143308639526, 'Rainy': 0.03201301768422127, 'Unknown Weather': 0.0004035258025396615}
✅ Accuracy for this image: 62.50%
Predicted Labels: ['Dry', 'Foggy']


In [ ]:

test_dataset = WeatherDataset(
    os.path.join(BASE_DIR, 'test'),
    os.path.join(BASE_DIR, 'test', '_classes.csv'),
    transform 
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [61]:
model.load_state_dict(torch.load(os.path.join(BASE_DIR, 'weather_model_efficientnetb1_multilabel.pt')))
model.to(DEVICE)
model.eval()


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [66]:
evaluate_model_metrics(model, val_loader, threshold=0.5)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



📊 Model Evaluation Metrics:
Overall Accuracy : 97.99%
Precision        : 96.23%
Recall           : 95.71%
F1-score         : 95.97%


(np.float64(0.9799107142857143),
 0.9622980251346499,
 0.9571428571428572,
 0.9597135183527306)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import torch

def evaluate_model_metrics(model, loader, threshold=0.5):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = torch.sigmoid(model(inputs))
            preds = (outputs > threshold).float()

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    overall_acc = (all_preds == all_labels).mean()


    precision = precision_score(all_labels, all_preds, average='micro')
    recall = recall_score(all_labels, all_preds, average='micro')
    f1 = f1_score(all_labels, all_preds, average='micro')

    print("\n📊 Model Evaluation Metrics:")
    print(f"Overall Accuracy : {overall_acc*100:.2f}%")
    print(f"Precision        : {precision*100:.2f}%")
    print(f"Recall           : {recall*100:.2f}%")
    print(f"F1-score         : {f1*100:.2f}%")

    return overall_acc, precision, recall, f1

evaluate_model_metrics(model, test_loader, threshold=0.5)



📊 Model Evaluation Metrics:
Overall Accuracy : 98.06%
Precision        : 95.94%
Recall           : 96.30%
F1-score         : 96.12%


(np.float64(0.9805555555555555),
 0.959409594095941,
 0.9629629629629629,
 0.9611829944547134)

In [ ]:

img_path = '/content/fog.jpeg'
labels = torch.tensor([[0,1,0,0,1,0,0,0]])  
input_tensor = transform(Image.open(img_path).convert('RGB')).unsqueeze(0).to(DEVICE)

model.eval()
with torch.no_grad():
    outputs = torch.sigmoid(model(input_tensor))
    preds = (outputs > 0.25).float() 

accuracy = (preds.cpu() == labels).float().mean().item()
print(f"✅ Model Accuracy on this image: {accuracy*100:.2f}%")


✅ Model Accuracy on this image: 75.00%
